Dependencies

In [ ]:
!pip install gymnasium[box2d]
!pip install swig
!pip install gymnasium[box2d] imageio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 103.6 MB/s eta 0:00:00


Exploring environment

In [ ]:
import gymnasium as gym

env = gym.make("LunarLander-v3")

In [ ]:

env.spec

EnvSpec(id='LunarLander-v3', entry_point='gymnasium.envs.box2d.lunar_lander:LunarLander', reward_threshold=200, nondeterministic=False, max_episode_steps=1000, order_enforce=True, disable_env_checker=False, kwargs={}, namespace=None, name='LunarLander', version=3, additional_wrappers=(), vector_entry_point=None)

In [ ]:
print(env.observation_space.low)

[ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ]


In [ ]:
print(env.observation_space)
print(env.observation_space.shape[0])

Box([ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ], [ 2.5        2.5       10.        10.         6.2831855 10.
  1.         1.       ], (8,), float32)
8


In [ ]:
print(env.action_space)
print(env.action_space.n)

Discrete(3)
3


In [ ]:
print(env.action_space.sample())
print(env.observation_space.sample())

3
[ 1.3810347   0.6009971  -1.8855329  -7.364891   -1.457443    4.1657453
  0.8680399   0.07304133]


In [ ]:
obs,info = env.reset()

In [ ]:
info

{}

In [ ]:
total_reward = 0
obs,info = env.reset()
for step in range(1000):
  action = env.action_space.sample()
  obs,reward,terminated,truncated,info=env.step(action)
  total_reward = total_reward + reward
  print(obs,reward,terminated,truncated,step)
  if terminated or truncated:
    break
print(total_reward)

[ 0.01496038  1.3951428   0.75660574 -0.36347583 -0.01714099 -0.16962102
  0.          0.        ] -1.021309789015703 False False 0
[ 0.02234554  1.3874038   0.7476031  -0.34408885 -0.02613104 -0.17981772
  0.          0.        ] 1.2046999860138612 False False 1
[ 0.02988405  1.3799744   0.76227057 -0.3303698  -0.03446109 -0.16661623
  0.          0.        ] -1.1840282487433342 False False 2
[ 0.03742275  1.3719456   0.76229453 -0.3570464  -0.0427905  -0.16660377
  0.          0.        ] -1.1473324621094037 False False 3
[ 0.04505224  1.363301    0.77367574 -0.38453802 -0.05340695 -0.21234863
  0.          0.        ] -2.4706446441021215 False False 4
[ 0.05260887  1.3540562   0.764525   -0.41121688 -0.06218233 -0.17552371
  0.          0.        ] -0.42383831431436536 False False 5
[ 0.06016579  1.3442124   0.7645488  -0.43789157 -0.07095759 -0.17552157
  0.          0.        ] -1.222530915300382 False False 6
[ 0.06786213  1.3349495   0.7780915  -0.4121105  -0.07933521 -0.1675678

Training DQN

In [ ]:
import gymnasium as gym
import numpy as np
import random
import copy
import imageio

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from collections import deque
import os


os.makedirs("results", exist_ok=True)
MODEL_DIR = "./results/models"
os.makedirs(MODEL_DIR, exist_ok=True)
MODEL_PATH = os.path.join(MODEL_DIR, "dqn_lunarlander.pt")

env = gym.make("LunarLander-v3", render_mode="rgb_array")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Q Network
class Q_Network(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Replay Buffer
class ReplayBuffer:
    def __init__(self, capacity, device):
        self.buffer = deque(maxlen=capacity)
        self.device = device

    def add(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        return (
            torch.tensor(states, dtype=torch.float32, device=self.device),
            torch.tensor(actions, dtype=torch.long, device=self.device),
            torch.tensor(rewards, dtype=torch.float32, device=self.device),
            torch.tensor(next_states, dtype=torch.float32, device=self.device),
            torch.tensor(dones, dtype=torch.float32, device=self.device),
        )

    def __len__(self):
        return len(self.buffer)


# Evaluate function
def evaluate_and_record(env, q_net, device, max_steps=1000):
    q_net.eval()
    frames = []

    state, _ = env.reset()

    total_reward = 0

    for _ in range(max_steps):
        frame = env.render()
        frames.append(frame)

        with torch.no_grad():
            q_vals = q_net(
                torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
            )
            action = torch.argmax(q_vals).item()

        next_state, reward, terminated, truncated, _ = env.step(action)


        total_reward += reward
        state = next_state

        if terminated or truncated:
            break

    q_net.train()
    return total_reward, np.array(frames)


q_net = Q_Network(
    input_size=env.observation_space.shape[0],
    hidden_size=64,
    output_size=env.action_space.n
).to(device)

target_q_net = copy.deepcopy(q_net).to(device)
target_q_net.eval()

optimizer = optim.Adam(q_net.parameters(), lr=1e-3)
loss_fn = nn.SmoothL1Loss()

buffer = ReplayBuffer(capacity=50_000, device=device)


# Hyper parameters
num_episodes = 200
batch_size = 32
gamma = 0.98

epsilon = 1.0
epsilon_min = 0.05
epsilon_decay = 0.995

target_update_every = 20
eval_every = 100

reward_list = []
epsilon_list = []
eval_rewards = []
eval_videos = []


# Train loop
for eps in range(num_episodes):
    state, _ = env.reset()
    total_reward = 0

    for step in range(1000):
        # Epsilon-greedy exploration
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            with torch.no_grad():
                q_vals = q_net(
                    torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
                )
                action = torch.argmax(q_vals).item()

        next_state, reward, terminated, truncated, _ = env.step(action)

        buffer.add(state, action, reward, next_state, terminated or truncated)
        state = next_state
        total_reward += reward

        if len(buffer) >= batch_size:
            states, actions, rewards, next_states, dones = buffer.sample(batch_size)

            with torch.no_grad():
                target_q = rewards + gamma * torch.max(
                    target_q_net(next_states), dim=1
                )[0] * (1 - dones)

            current_q = q_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)
            loss = loss_fn(current_q, target_q)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if terminated or truncated:
            break

    # Target network update
    if eps % target_update_every == 0:
        target_q_net.load_state_dict(q_net.state_dict())

    # Epsilon decay
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    reward_list.append(total_reward)
    epsilon_list.append(epsilon)

    print(f"Episode {eps:4d} | Reward: {total_reward:7.2f} | Epsilon: {epsilon:.3f}")

    # Evaluation
    if eps % eval_every == 0:
        eval_reward, frames = evaluate_and_record(env, q_net, device)
        eval_rewards.append(eval_reward)
        eval_videos.append(frames)

        print(f"[EVAL] Episode {eps} | Eval reward: {eval_reward:.2f}")

        video_path = f"./results/lander_eval_ep{eps}.mp4"
        imageio.mimsave(video_path, frames, fps=30)
        print(f"[VIDEO SAVED] {video_path}")

        torch.save(
        {
            "episode": eps,
            "model_state_dict": q_net.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "epsilon": epsilon,
            "eval_reward": eval_reward,
        },
        MODEL_PATH
        )

env.close()


Device: cuda


/tmp/ipython-input-684504018.py:66: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  torch.tensor(states, dtype=torch.float32, device=self.device),


Episode    0 | Reward: -241.18 | Epsilon: 0.995
[EVAL] Episode 0 | Eval reward: -669.25


[VIDEO SAVED] ./results/lander_eval_ep0.mp4
Episode    1 | Reward:  -79.37 | Epsilon: 0.990
Episode    2 | Reward: -203.36 | Epsilon: 0.985
Episode    3 | Reward: -100.23 | Epsilon: 0.980
Episode    4 | Reward: -108.37 | Epsilon: 0.975
Episode    5 | Reward: -268.84 | Epsilon: 0.970
Episode    6 | Reward: -249.35 | Epsilon: 0.966
Episode    7 | Reward: -108.52 | Epsilon: 0.961
Episode    8 | Reward: -416.53 | Epsilon: 0.956
Episode    9 | Reward: -160.31 | Epsilon: 0.951
Episode   10 | Reward: -105.42 | Epsilon: 0.946
Episode   11 | Reward: -214.12 | Epsilon: 0.942
Episode   12 | Reward: -101.53 | Epsilon: 0.937
Episode   13 | Reward: -113.20 | Epsilon: 0.932
Episode   14 | Reward: -205.45 | Epsilon: 0.928
Episode   15 | Reward: -205.11 | Epsilon: 0.923
Episode   16 | Reward: -172.56 | Epsilon: 0.918
Episode   17 | Reward: -304.20 | Epsilon: 0.914
Episode   18 | Reward: -120.18 | Epsilon: 0.909
Episode   19 | Reward: -101.73 | Epsilon: 0.905
Episode   20 | Reward: -118.28 | Epsilon: 0.

[EVAL] Episode 100 | Eval reward: -208.30
[VIDEO SAVED] ./results/lander_eval_ep100.mp4
Episode  101 | Reward:  -25.66 | Epsilon: 0.600
Episode  102 | Reward: -161.24 | Epsilon: 0.597
Episode  103 | Reward:  -22.47 | Epsilon: 0.594
Episode  104 | Reward:  -38.06 | Epsilon: 0.591
Episode  105 | Reward:  -86.07 | Epsilon: 0.588
Episode  106 | Reward:  -69.64 | Epsilon: 0.585
Episode  107 | Reward: -101.36 | Epsilon: 0.582
Episode  108 | Reward:  -11.95 | Epsilon: 0.579
Episode  109 | Reward: -105.43 | Epsilon: 0.576
Episode  110 | Reward:   33.06 | Epsilon: 0.573
Episode  111 | Reward:  -54.53 | Epsilon: 0.570
Episode  112 | Reward: -104.86 | Epsilon: 0.568
Episode  113 | Reward: -113.93 | Epsilon: 0.565
Episode  114 | Reward:   60.38 | Epsilon: 0.562
Episode  115 | Reward:  -46.40 | Epsilon: 0.559
Episode  116 | Reward: -210.62 | Epsilon: 0.556
Episode  117 | Reward:  -28.70 | Epsilon: 0.554
Episode  118 | Reward:  -53.29 | Epsilon: 0.551
Episode  119 | Reward:  -53.94 | Epsilon: 0.548


...

[EVAL] Episode 1500 | Eval reward: 266.92
[VIDEO SAVED] ./results/lander_eval_ep1500.mp4
Episode 1501 | Reward:  232.53 | Epsilon: 0.050
Episode 1502 | Reward:  183.74 | Epsilon: 0.050
Episode 1503 | Reward:  142.73 | Epsilon: 0.050
Episode 1504 | Reward:   21.66 | Epsilon: 0.050
Episode 1505 | Reward:  274.48 | Epsilon: 0.050
Episode 1506 | Reward:  -24.00 | Epsilon: 0.050
Episode 1507 | Reward:    4.81 | Epsilon: 0.050
Episode 1508 | Reward:    5.19 | Epsilon: 0.050
Episode 1509 | Reward:  246.60 | Epsilon: 0.050
Episode 1510 | Reward:  270.60 | Epsilon: 0.050
Episode 1511 | Reward:  211.65 | Epsilon: 0.050
Episode 1512 | Reward:   14.89 | Epsilon: 0.050
Episode 1513 | Reward:   65.96 | Epsilon: 0.050
Episode 1514 | Reward:  -12.22 | Epsilon: 0.050
Episode 1515 | Reward:   13.65 | Epsilon: 0.050
Episode 1516 | Reward:  -35.51 | Epsilon: 0.050
Episode 1517 | Reward:   -3.36 | Epsilon: 0.050
Episode 1518 | Reward:  124.60 | Epsilon: 0.050
Episode 1519 | Reward:  270.42 | Epsilon: 0.050

[EVAL] Episode 1600 | Eval reward: -39.52
[VIDEO SAVED] ./results/lander_eval_ep1600.mp4
Episode 1601 | Reward:   38.15 | Epsilon: 0.050
Episode 1602 | Reward:   20.84 | Epsilon: 0.050
Episode 1603 | Reward:  -34.96 | Epsilon: 0.050
Episode 1604 | Reward:   12.32 | Epsilon: 0.050
Episode 1605 | Reward: -148.78 | Epsilon: 0.050
Episode 1606 | Reward:   31.44 | Epsilon: 0.050
Episode 1607 | Reward:  -17.07 | Epsilon: 0.050
Episode 1608 | Reward:  -70.55 | Epsilon: 0.050
Episode 1609 | Reward:  272.57 | Epsilon: 0.050
Episode 1610 | Reward:  296.28 | Epsilon: 0.050
Episode 1611 | Reward:   24.97 | Epsilon: 0.050
Episode 1612 | Reward:  219.63 | Epsilon: 0.050
Episode 1613 | Reward:   -4.95 | Epsilon: 0.050
Episode 1614 | Reward:  214.94 | Epsilon: 0.050
Episode 1615 | Reward:  217.99 | Epsilon: 0.050
Episode 1616 | Reward:  -50.33 | Epsilon: 0.050
Episode 1617 | Reward:   13.19 | Epsilon: 0.050
Episode 1618 | Reward:   35.32 | Epsilon: 0.050
Episode 1619 | Reward:   50.40 | Epsilon: 0.050

[EVAL] Episode 1700 | Eval reward: 225.59
[VIDEO SAVED] ./results/lander_eval_ep1700.mp4
Episode 1701 | Reward:  218.32 | Epsilon: 0.050
Episode 1702 | Reward:  -39.02 | Epsilon: 0.050
Episode 1703 | Reward:  -10.82 | Epsilon: 0.050
Episode 1704 | Reward: -154.11 | Epsilon: 0.050
Episode 1705 | Reward:   -7.70 | Epsilon: 0.050
Episode 1706 | Reward:  -31.38 | Epsilon: 0.050
Episode 1707 | Reward:    2.18 | Epsilon: 0.050
Episode 1708 | Reward:   16.06 | Epsilon: 0.050
Episode 1709 | Reward:   32.77 | Epsilon: 0.050
Episode 1710 | Reward:  227.30 | Epsilon: 0.050
Episode 1711 | Reward:  211.77 | Epsilon: 0.050
Episode 1712 | Reward:   35.58 | Epsilon: 0.050
Episode 1713 | Reward:   39.34 | Epsilon: 0.050
Episode 1714 | Reward:  239.97 | Epsilon: 0.050
Episode 1715 | Reward:  252.65 | Epsilon: 0.050
Episode 1716 | Reward:   27.80 | Epsilon: 0.050
Episode 1717 | Reward:  264.32 | Epsilon: 0.050
Episode 1718 | Reward:  226.10 | Epsilon: 0.050
Episode 1719 | Reward:   -9.35 | Epsilon: 0.050

[EVAL] Episode 1800 | Eval reward: -36.23
[VIDEO SAVED] ./results/lander_eval_ep1800.mp4
Episode 1801 | Reward:   39.48 | Epsilon: 0.050
Episode 1802 | Reward:  257.05 | Epsilon: 0.050
Episode 1803 | Reward:  254.15 | Epsilon: 0.050
Episode 1804 | Reward:  176.87 | Epsilon: 0.050
Episode 1805 | Reward:  -11.39 | Epsilon: 0.050
Episode 1806 | Reward:   -3.59 | Epsilon: 0.050
Episode 1807 | Reward:   10.94 | Epsilon: 0.050
Episode 1808 | Reward:  -11.37 | Epsilon: 0.050
Episode 1809 | Reward:  224.22 | Epsilon: 0.050
Episode 1810 | Reward:  223.10 | Epsilon: 0.050
Episode 1811 | Reward:  253.46 | Epsilon: 0.050
Episode 1812 | Reward:  273.93 | Epsilon: 0.050
Episode 1813 | Reward:  -52.26 | Epsilon: 0.050
Episode 1814 | Reward:   24.78 | Epsilon: 0.050
Episode 1815 | Reward:    9.84 | Epsilon: 0.050
Episode 1816 | Reward:  247.56 | Epsilon: 0.050
Episode 1817 | Reward:   -1.76 | Epsilon: 0.050
Episode 1818 | Reward:  -19.94 | Epsilon: 0.050
Episode 1819 | Reward:  -48.55 | Epsilon: 0.050

[EVAL] Episode 1900 | Eval reward: 115.66
[VIDEO SAVED] ./results/lander_eval_ep1900.mp4
Episode 1901 | Reward:  279.36 | Epsilon: 0.050
Episode 1902 | Reward:  260.59 | Epsilon: 0.050
Episode 1903 | Reward:   17.37 | Epsilon: 0.050
Episode 1904 | Reward: -135.44 | Epsilon: 0.050
Episode 1905 | Reward:  236.43 | Epsilon: 0.050
Episode 1906 | Reward:   -6.40 | Epsilon: 0.050
Episode 1907 | Reward:  231.13 | Epsilon: 0.050
Episode 1908 | Reward:  181.04 | Epsilon: 0.050
Episode 1909 | Reward:  263.50 | Epsilon: 0.050
Episode 1910 | Reward:   19.41 | Epsilon: 0.050
Episode 1911 | Reward:  -38.18 | Epsilon: 0.050
Episode 1912 | Reward:  236.75 | Epsilon: 0.050
Episode 1913 | Reward:   31.45 | Epsilon: 0.050
Episode 1914 | Reward:  252.92 | Epsilon: 0.050
Episode 1915 | Reward:   -6.06 | Epsilon: 0.050
Episode 1916 | Reward:   40.03 | Epsilon: 0.050
Episode 1917 | Reward:  221.37 | Epsilon: 0.050
Episode 1918 | Reward:  189.81 | Epsilon: 0.050
Episode 1919 | Reward:  245.41 | Epsilon: 0.050

In [ ]:
!zip -r results.zip results
from google.colab import files
files.download("results.zip")

  adding: results/ (stored 0%)
  adding: results/lander_eval_ep1300.mp4 (deflated 15%)
  adding: results/lander_eval_ep200.mp4 (deflated 7%)
  adding: results/lander_eval_ep600.mp4 (deflated 9%)
  adding: results/lander_eval_ep1400.mp4 (deflated 12%)
  adding: results/lander_eval_ep1700.mp4 (deflated 10%)
  adding: results/lander_eval_ep0.mp4 (deflated 6%)
  adding: results/lander_eval_ep300.mp4 (deflated 7%)
  adding: results/lander_eval_ep1100.mp4 (deflated 13%)
  adding: results/lander_eval_ep900.mp4 (deflated 11%)
  adding: results/lander_eval_ep800.mp4 (deflated 11%)
  adding: results/lander_eval_ep400.mp4 (deflated 10%)
  adding: results/lander_eval_ep700.mp4 (deflated 11%)
  adding: results/lander_eval_ep1600.mp4 (deflated 10%)
  adding: results/models/ (stored 0%)
  adding: results/models/dqn_lunarlander.pt (deflated 21%)
  adding: results/lander_eval_ep1500.mp4 (deflated 12%)
  adding: results/lander_eval_ep1800.mp4 (deflated 12%)
  adding: results/lander_eval_ep100.mp4 (defla

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>